# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

## the csv files has been created in current folder, this part of the code can be skipped.

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)
print(len(file_path_list))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [75]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [2]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

This should make a connection to a Cassandra instance your local machine, generate a session to execute queries 

This program require cassandra package

In [3]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

Generate a keyspace called sparkify if not exists in apache cassandra

In [4]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")
    
except Exception as e:
    print("error")


#### Set Keyspace

In [5]:
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Question 1. We need to provide the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

##### Step 1.
Create a table called songInfo if not exists. This table has sessionId (varint), itemInSession (varint), artist (text), firstName (text), song (text), length (float). To select record with sessionId = 338, and itemInSession = 4, we choose sessionid and itemInSession as partition keys.

##### Step 2.
Import the data from .csv file. using INSERT query to insert corresponding data from the .csv files. Following the order of the table. change the data type of input data if necessary.

##### Step 3.
Using SELECT queries to get artist, song, length for the table for userid = 10 and sessionid = 182

In [8]:
create1 = "CREATE TABLE IF NOT EXISTS songInfo "
createquery1 = create1 + "(sessionId varint, itemInSession varint, artist text, firstName text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(createquery1)
except Exception as e:
    print("False to create the songInfo")

#### Uncomment and Run the block below to drop the table only if mistakes happened in insert. Then modify and create the table.

In [7]:
#session.execute("DROP TABLE IF EXISTS songInfo")

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## add the information to the table
        #print(line)
        query = "INSERT INTO songInfo (sessionId, itemInSession, artist, firstName, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[1], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Using select to exam if the table is contain right information.
result1 = session.execute("""SELECT artist, song, length FROM songInfo WHERE sessionId = 338 AND itemInSession = 4""")

for result in result1:
    print(result)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


#### Question 2. We need to list artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182. 

##### Step 1.
Create a table called listenlist if not exists. This table has sessionId (varint), userId (varint), itemInSession (varint), artist (text), song (text), firstName (text), lastName (text). We need to use userid and sessionid to perform select, so userid and sessionid are chosen as partition key. and the result should be sorted by itemInSession. So we set the itemInSession as clustering column. 

##### Step 2.
Import the data from .csv file. using INSERT query to insert corresponding data from the .csv files. Following the order of the table. change the data type of input data if necessary.

##### Step 3.
Using SELECT queries to get artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Uncomment and Run the block below to drop the table only if mistakes happened in insert. Then modify and create the table.

In [19]:
#session.execute("DROP TABLE IF EXISTS listenlist")

In [11]:
create2 = "CREATE TABLE IF NOT EXISTS listenlist "
createquery2 = create2 + "(sessionId varint, userId varint, itemInSession varint, artist text, song text, \
                        firstName text, lastName text, PRIMARY KEY ((sessionId, userId), itemInSession))"
try:
    session.execute(createquery2)
except Exception as e:
    print("False to create the listenlist")
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
        query = "INSERT INTO listenlist (sessionId, userId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))
        
result1 = session.execute("""SELECT artist, song, firstName, lastName FROM listenlist \
                            WHERE sessionId = 182 AND userId = 10""")

for result in result1:
    print(result)                    

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


#### Question 3. We need to select every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

##### Step 1.
Create a table called listener if not exists. This table has song (text), userId (varint), firstName (text), lastName (text). To select the listener of a certain song, we need to select the song and userId as the partition key. 

##### Step 2.
Import the data from .csv file. using INSERT query to insert corresponding data from the .csv files. Following the order of the table. change the data type of input data if necessary.

##### Step 3.
Using SELECT queries to get firstName, lastName for the listener of All Hands Against His Own.

#### Uncomment and Run the block below to drop the table only if mistakes happened in insert. Then modify and create the table.

In [12]:
#session.execute("DROP TABLE IF EXISTS listener")

In [13]:
create3 = "CREATE TABLE IF NOT EXISTS listener "
createquery3 = create3 + "(song text, userId varint, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(createquery3)
except Exception as e:
    print("False to create the listener")
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
        query = "INSERT INTO listener (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        
result1 = session.execute("""SELECT firstName, lastName FROM listener WHERE song = 'All Hands Against His Own'""")

for result in result1:
    print(result)     

                    

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [14]:
# queries to drop the tables

session.execute("DROP TABLE IF EXISTS songInfo")
session.execute("DROP TABLE IF EXISTS listenlist")
session.execute("DROP TABLE IF EXISTS listener")

### Close the session and cluster connection¶

In [15]:
# shutdown the session and the cluster
session.shutdown()
cluster.shutdown()